In [1]:
import numpy as np
import cv2
import math
from scipy import ndimage
from PIL import Image 
from PIL import ImageEnhance

In [3]:
xyz= input("Enter Image Name ")

Enter Image Name image_5.jpg


In [4]:
#Angle
img_before = cv2.imread(xyz)

#cv2.imshow("Before", img_before)    
key = cv2.waitKey(0)

img_gray = cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY)
img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)

angles = []

for x1, y1, x2, y2 in lines[0]:
    cv2.line(img_before, (x1, y1), (x2, y2), (255, 0, 0), 3)
    angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
    angles.append(angle)

median_angle = np.median(angles)
img_rotated = ndimage.rotate(img_before, median_angle)

print ("Angle is {}".format(median_angle))

Angle is 0.0


In [5]:
nameim = input('Enter new name for image ')

Enter new name for image Zmid1.jpg


In [6]:
cv2.imwrite(nameim,img_rotated)

True

In [7]:
#DPI
image_file = Image.open(nameim)
print("Image data: ")
print("Format:" + image_file.format)
width, height  = image_file.size
print("Width: " + str(width))
print("Height: " + str(height))
if image_file.info.get('dpi'):
    x_dpi, y_dpi = image_file.info['dpi']
    if x_dpi != y_dpi:
        print('Inconsistent DPI image data')
    print("x_dpi: " + str(x_dpi))
    print("y_dpi: " + str(y_dpi))
else:
    print('No DPI data. Invalid Image header')
    print(image_file.info)

Image data: 
Format:JPEG
Width: 3024
Height: 2024
No DPI data. Invalid Image header
{'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1)}


In [8]:
#blurscore
im = Image.open(nameim).convert('L') # to grayscale
array = np.asarray(im, dtype=np.int32)

gy, gx = np.gradient(array)
gnorm = np.sqrt(gx**2 + gy**2)
sharpness = np.average(gnorm)
blurscr=(10*sharpness)
print(blurscr)

62.77304825632109


In [9]:
if (blurscr<100):
    if(blurscr<10):
        sha=100
    elif(blurscr<50):
        sha=40
    elif(blurscr<75):
        sha=10
    elif(blurscr<99):
        sha=5
    else:
        sha=1     

In [10]:
img = Image.open(nameim)
enhd = ImageEnhance.Sharpness(img)
enhd = enhd.enhance(sha)

In [11]:
saveimg = input("Enter a name again to save ")

Enter a name again to save Zmid2.jpg


In [12]:
enhd.save(saveimg)

In [13]:
#noise
image_file = Image.open(nameim) # open colour image
image_file= image_file.convert('L')

In [14]:
def noise(img):
    # load color image
    im = cv2.imread(img)
# smooth the image with alternative closing and opening
# with an enlarging kernel
    morph = im.copy()
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
# take morphological gradient
#gradient_image = cv2.morphologyEx(morph, cv2.MORPH_GRADIENT, kernel)
# split the gradient image into channels
    image_channels = np.split(np.asarray(morph), 3, axis=2)
    channel_height, channel_width, _ = image_channels[0].shape
# apply Otsu threshold to each channel
    for i in range(0, 3):
        _, image_channels[i] = cv2.threshold(image_channels[i], 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
        image_channels[i] = np.reshape(image_channels[i], newshape=(channel_height, channel_width, 1))
# merge the channels
    image_channels = np.concatenate((image_channels[0], image_channels[1], image_channels[2]), axis=2)
# save the denoised image
    cv2.imwrite('output.jpg', image_channels)
    


In [15]:
noise(saveimg)